In [1]:
import json
import nltk

with open("gists/minidogs.jsonl", "r") as inf:
    lns = [json.loads(i) for i in inf]

In [2]:
ROOT = 'ROOT'
def getNodes(parent, out):
    for node in parent:
        if type(node) is nltk.Tree:
            if node.label() == ROOT:
                print("======== Sentence =========")
                print("Sentence:", " ".join(node.leaves()))
            else:
                #print("Label:", node.label())
                #print("Leaves:", node.leaves())
                if len(node.leaves()) > 1:
                    out.append((node.label(), " ".join(node.leaves())))

            getNodes(node, out)
        else:
            pass

In [3]:
from environments.envs import ENVIRONMENTS
import argparse
import kenlm
import json

from klm.query import LM
from klm.query import get_unigram_probs

env = ENVIRONMENTS["DEV"]
LOC = env['klm_model']
UG_MODEL = env["ug_model"]

lm = LM(loc=LOC)
up = get_unigram_probs(UG_MODEL)
from klm.query import slor

In [123]:
from nltk.tree import *
from collections import defaultdict

label2slor = defaultdict(list)

constit_dfs = []

for l in lns:
    out = list()
    tree2 = Tree.fromstring(l["parse"])
    getNodes(tree2, out)
    slabels = set()
    for o in out:
        label, str_ = o
        slabels.add(label)
        label2slor[label].append(slor(str_,lm, up))
    for s in slabels:
        constit_dfs.append(s)
        
    
import numpy as np

l2nlor = [(l, np.mean(label2slor[l])) for l in label2slor]
l2nlor.sort(key=lambda x:x[1], reverse=True)


from collections import Counter
constits = [o for o in Counter(constit_dfs).most_common()]
constits

[('VP', 150),
 ('S', 150),
 ('NP', 141),
 ('PP', 105),
 ('SBAR', 70),
 ('ADJP', 43),
 ('ADVP', 18),
 ('FRAG', 17),
 ('PRN', 13),
 ('SQ', 7),
 ('UCP', 6),
 ('SBARQ', 3),
 ('WHNP', 3),
 ('QP', 2),
 ('INTJ', 2),
 ('NP-TMP', 2),
 ('LST', 1),
 ('SINV', 1),
 ('WHPP', 1)]

In [115]:
def find_ngrams(input_list, n):
    return zip(*[input_list[i:] for i in range(n)])

index = 26
all_ = []
for o in find_ngrams([o["word"] for o in lns[index]['tokens']], n=6):
    str_ = " ".join(o)
    all_.append((str_, max(slor(str_,lm, up), 0)))
    
all_.sort(key=lambda x:x[1], reverse=True)

for o in all_:
    print(o)

('thirty min walks a day .', 0.9980161390218095)
('My little couch potato dog needs', 0.6113157377929687)
('3 thirty min walks a day', 0.4164939557576499)
('little couch potato dog needs 3', 0.3197258045450842)
('couch potato dog needs 3 thirty', 0.031799057637532734)
('potato dog needs 3 thirty min', 0.03132666356608086)
('dog needs 3 thirty min walks', 0)
('needs 3 thirty min walks a', 0)


In [107]:
toks = "my little couch potato needs 3 thirty minute walks".split()

def insert_all_ellipses(toks, start_index, end_index):
    '''
    return all spans of toks with elipeses removed
    
    This is quadratic b/c you loop over start and ends of elipses. This cuts
    down on the search complexity of the problem
    
    start_index(int): where to start looping, to insert ellipse
    end_index(int): where to end looping, to insert ellipse
         - If you are looping over the whole sentences then 
             - start_index = 0 and end_index = len(toks)
         - But also you might loop over the last 5 tokens in the sentence
         
    You could make more recursive calls to add more ellipses
    
    Returns a list of tuples. 
        - The first item in the tuple is the tokens included 
        - The second item in the tuple is the start of ellipse
        - The third item in the tuple is the end of ellipse
    '''
    for start_ellipse in range(start_index, end_index):
        for end_ellipse in range(start_ellipse + 1, end_index):
            yield(start_ellipse, end_ellipse)
    
opts = []

from collections import defaultdict

classes = defaultdict(list)

for i in insert_all_ellipses(toks, start_index=0, end_index=len(toks) + 1):
    ellipse_min, ellipse_max = i
    printthis = ""
    for tno, t in enumerate(toks):
        if tno == ellipse_min:
            printthis = printthis + " ... "
        elif tno > ellipse_min and tno < ellipse_max:
            pass
        else:
            printthis = printthis + " " + t
    str_ = printthis.replace(" ... ", " ").strip().replace("  ", " ")
    classes[len(str_.split())].append((printthis, slor(str_,lm, up)))
    
for c in classes:
    print("{}".format(c))
    opts = classes[c]
    opts.sort(key=lambda x:x[1], reverse=True)
    opts[0:20]
    for o in opts:
        print(o)

8
(' my little couch potato needs 3 thirty minute ... ', 0.6129066852813718)
(' my little couch potato ...  3 thirty minute walks', 0.5386765264556885)
(' my little couch potato needs ...  thirty minute walks', 0.5155214225158691)
(' my little couch potato needs 3 ...  minute walks', 0.3912193690338137)
(' my ...  couch potato needs 3 thirty minute walks', 0.3645562493133543)
(' my little couch potato needs 3 thirty ...  walks', 0.3547105074310304)
(' my little couch ...  needs 3 thirty minute walks', 0.3040491313323983)
(' ...  little couch potato needs 3 thirty minute walks', 0.30112789797973605)
(' my little ...  potato needs 3 thirty minute walks', 0.19168549163208048)
7
(' my little couch potato ...  thirty minute walks', 0.6974210636648996)
(' my little couch potato needs 3 ...  walks', 0.5883220816476005)
(' my little couch potato needs 3 thirty ... ', 0.37479685377022876)
(' my little couch potato needs ...  minute walks', 0.308935251831055)
(' ...  couch potato needs 3 thirty 

In [98]:
slor("my little couch potato thirty minute walks",lm, up)
lm.model.score("my little couch potato needs walks", bos = True, eos = True)


[(u, up[u]) for u in "my little couch potato needs walks".split()]

[('my', -3.6942735),
 ('little', -3.8233995),
 ('couch', -4.7277107),
 ('potato', -4.621139),
 ('needs', -3.7470686),
 ('walks', -4.2313447)]

In [12]:
out = []
opts = []
K = 20
sno = 26
tree2 = Tree.fromstring(lns[sno]["parse"])
getNodes(tree2, out)
for o in out:
    label, frag = o
    if len(frag.split()) < K:
        opts.append((frag, slor(frag,lm, up)))

opts = list(set(opts))
opts.sort(key=lambda x:x[1],reverse=True)

print(" ".join([o["word"] for o in lns[sno]["tokens"]]))
for o in opts:
    print(o)

My little couch potato dog needs 3 thirty min walks a day .
('My little couch potato dog needs 3 thirty min walks a day .', 0.8180272019343447)
('My little couch potato dog needs 3 thirty min', 0.7242806523708762)
('My little couch potato dog', 0.6557964061962892)
('walks a day', 0.09627045926513667)
('3 thirty min', 0.08536011938476544)
('a day', -0.09840599101257297)
('needs 3 thirty min', -0.1275434680114751)
('3 thirty', -1.6068268827514647)


In [131]:

sno = 17

out = list()
tree2 = Tree.fromstring(lns[sno]["parse"])
getNodes(tree2, out)

out

[('S',
  "`` The first thing I thought was ` shes in heat ' , but since that 's not it ... check and see if her vulva looks red or swollen or any discharge , I had a male that was doing that to a female and it was because she had a yeast infection ."),
 ('S',
  "The first thing I thought was ` shes in heat ' , but since that 's not it ... check and see if her vulva looks red or swollen or any discharge , I had a male that was doing that to a female"),
 ('NP',
  "The first thing I thought was ` shes in heat ' , but since that 's not it ... check and see if her vulva looks red or swollen or any discharge"),
 ('NP', 'The first thing'),
 ('SBAR',
  "I thought was ` shes in heat ' , but since that 's not it ... check and see if her vulva looks red or swollen or any discharge"),
 ('S',
  "I thought was ` shes in heat ' , but since that 's not it ... check and see if her vulva looks red or swollen or any discharge"),
 ('VP',
  "thought was ` shes in heat ' , but since that 's not it ... check

In [150]:
tm = [p[1] for p in out]
import itertools
co = 0
opts = []
max_c = 100

for i in range(1,4):
    for v in itertools.combinations(tm, r=i):
        co += 1
        if sum([len(i) for i in v]) < max_c:
            opts.append(v)
    
opts[-10:]

[('was because she had a yeast infection',
  'because she had a yeast infection',
  'she had a yeast infection'),
 ('was because she had a yeast infection',
  'because she had a yeast infection',
  'had a yeast infection'),
 ('was because she had a yeast infection',
  'because she had a yeast infection',
  'a yeast infection'),
 ('was because she had a yeast infection',
  'she had a yeast infection',
  'had a yeast infection'),
 ('was because she had a yeast infection',
  'she had a yeast infection',
  'a yeast infection'),
 ('was because she had a yeast infection',
  'had a yeast infection',
  'a yeast infection'),
 ('because she had a yeast infection',
  'she had a yeast infection',
  'had a yeast infection'),
 ('because she had a yeast infection',
  'she had a yeast infection',
  'a yeast infection'),
 ('because she had a yeast infection',
  'had a yeast infection',
  'a yeast infection'),
 ('she had a yeast infection', 'had a yeast infection', 'a yeast infection')]